 Прочитайте csv файл, который находится по этой ссылке, при помощи библиотеки pandas

https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ti6Su94955DZ4Tky8EbwifpgZf_dTjpBdiVH0Ukhsq94jZdqoHuUytZsFZKfwpXEUCKRFteJRc9P/pub?gid=889004448&single=true&output=csv
2. В данных вы найдете информацию о событиях, которые произошли с объявлением 121288 за два дня. Рассчитайте следующие метрики в разрезе каждого дня:

количество показов
количество кликов
CTR 
сумма потраченных денег 
То есть для каждой метрики у вас должно получиться два числа - за 2019-04-01 и 2019-04-02

Рассчитать сумму потраченных денег можно по следующей формуле - разделите значение из колонки ad_cost на 1000 и умножьте на количество показов объявления

3. Теперь найдите процентную разницу между этими метриками. То есть найдите, насколько процентов каждая метрика увеличилась/уменьшилась 2 апреля по сравнению с 1 апреля

4. Создайте текстовый файл, в котором будет собрана информация о том, какие метрики наблюдаются 2 апреля (представьте, что 2 апреля это сегодня, условно, вы отправляете отчет за сегодня, сравнивая данные со вчера), а также, на сколько процентов они уменьшились по сравнению со вчера. То есть в текстовом файле должен быть список метрик, их значения 2 апреля и процентное отличие от 1 апреля. Вариант текстового отчета:

Отчет по объявлению 121288 за 2 апреля
Траты: Х рублей (Y%)
Показы: X (Y%)
Клики: X (-Y%)
CTR: X (-Y%)

5. Отправьте получившийся текст к себе в личные сообщения во ВКонтакте в виде сообщения со сводкой метрик

6. Все предыдущие шаги оформите в виде исполняемого скрипта (скриптов) и скрипта для DAG'а, в котором при помощи BashOperator'а (или других операторов на ваше усмотрение) будет вызываться исполняемый скрипт (скрипты)

7. В расписании для крона укажите каждый понедельник в 12 утра. Так, чтобы ваш скрипт с расчётом дневных метрик отправлялся вам в личку каждый понедельник в 12 утра из Airflow

In [6]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime

default_args = {
    'owner': 'izhusupov',                  
    'depends_on_past': False,  
    'start_date' : datetime(2020,12,1),           
    'retries': 0                         
}
 
dag = DAG('Zhusupov_first_dag',        
    default_args=default_args, 
    catchup=False,              
    schedule_interval='0 12 * * 1')


def report_to_vk():
    
import pandas as pd
import vk_api
import random
import requests
import json

# read_file
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ti6Su94955DZ4Tky8EbwifpgZf_dTjpBdiVH0Ukhsq94jZdqoHuUytZsFZKfwpXEUCKRFteJRc9P/pub?gid=889004448&single=true&output=csv')

#need_columns
df1 = ( df.groupby(['date', 'event', 'ad_cost'], as_index = False)
    .agg({'ad_id': 'count'})
    .pivot(index = ['date', 'ad_cost'], columns = 'event', values = 'ad_id')
    .reset_index() )

# add_columns_CTR_and_total_spent_money
df2 = df1.assign(CTR = round(df1.click / df1.view * 100, 2) , total_spent_money = round(df1.ad_cost / 1000 * df1.view,2)).T.reset_index()

#first_index_as columns_and_change_type_of_columns
df2.columns = df2.iloc[0]
df2 = df2[2:]
df2 = df2.astype({'2019-04-01':'float64', '2019-04-02':'float64'})

# add_columns_difference_from_the_previous_day
df2 = df2.assign(difference_from_the_previous_day = round((df2['2019-04-02'] / df2['2019-04-01'] - 1)*100))
#delete_needless_column_and_change_type
del df2['2019-04-01']
df2 = df2.astype({'2019-04-02':'int64', 'difference_from_the_previous_day':'int64'})

#write_information_to_the_file
k = []
k.append(f"Отчет по объявлению 121288 за {df2.columns[1]}:\n")
for i,rows in df2.iterrows():
    if rows['date'] =='total_spent_money':
        k.append(f"{rows['date']}: {rows['2019-04-02']} рублей ({rows['difference_from_the_previous_day']} %)\n")
    else:
        k.append(f"{rows['date']}: {rows['2019-04-02']} ({rows['difference_from_the_previous_day']} %)\n")
f.close()

# Token which you obtained via vk
app_token = ''

# id of the 1st chat
chat_id = 78

# id of my user-receiver
my_id = 151391097

# Initialize session
vk_session = vk_api.VkApi(token=app_token)

# Make it possible to use vk api methods as python methods
vk = vk_session.get_api()

for i in k:
    vk.messages.send(
    chat_id=1,
    random_id=random.randint(1, 2 ** 31),
    message= i)

    
t1 = PythonOperator(
    task_id='send_report_text_to_vk',
    python_callable=report_to_vk,
    dag=dag)
